# Prvi zadatak
Svjetska Banka želi da donese odluku o tome da li je novopredloženi projekat isplativ za realizaciju:
- Na osnovu slučajeva iz prošlosti prikazani na slici 1 treba napraviti klasifikacioni model koristeći algoritam Naivnog Bajesa.
- Istu tabelu iskoristiti i primjeniti stablo odlučivanja.

U oba slučaja provjeriti koja će se donijeti odluka.

<img src="slika1.png" alt="Slika 1" width="300"/>

## Import potrebnih biblioteka

Importovanje potrebnih biblioteka koje će biti potrebne za obradu podataka i implementaciju modela.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.metrics import accuracy_score

## Organizacija podataka
Definišu se ulazni podaci kao Python dictionary i konvertuju se u pandas DataFrame.

1. Definicija podataka:
Podaci se nalaze u rječniku (data), gdje su ključevi nazivi kolona, a vrijednosti su liste koje predstavljaju podatke u tim kolonama. Svaka lista sadrži niz vrijednosti, a broj elemenata u svakoj listi mora biti isti, jer svaka lista predstavlja jednu kolonu u DataFrame-u.

2. Kreiranje DataFrame-a:
Koristi pandas.DataFrame() funkciju da kreira DataFrame objekat. DataFrame je tabularna struktura podataka u Pandas-u.

Kada se pozove pd.DataFrame(data), Pandas će:
- Koristiti ključeve rječnika kao nazive kolona (na primjer: "Ukupan rizik", "Vremenski horizont", itd.).
- Svaku od lista u rječniku pretvoriti u redove podataka za svaku kolonu.


In [4]:
data = {
    "Ukupan rizik": ["nizak", "visok", "nizak", "visok", "nizak", "visok", "nizak", "visok", "nizak", "visok", "nizak", "visok", "nizak", "visok", "nizak"],
    "Vremenski horizont": ["kratak rok", "srednji rok", "dugi rok", "kratak rok", "srednji rok", "dugi rok", "kratak rok", "srednji rok", "dugi rok", "kratki rok", "srednji rok", "dugi rok", "kratki rok", "srednji rok", "dugi rok"],
    "Ukupni troskovi": ["visoki", "niski", "visoki", "niski", "visoki", "niski", "visoki", "niski", "visoki", "niski", "visoki", "niski", "visoki", "niski", "visoki"],
    "Drustvena korist": ["visoka", "niska", "visoka", "visoka", "niska", "niska", "visoka", "visoka", "visoka", "niska", "visoka", "niska", "visoka", "niska", "visoka"],
    "Realizovati?": ["ne", "ne", "da", "da", "ne", "ne", "da", "da", "ne", "ne", "da", "da", "ne", "ne", "da"]
}

df = pd.DataFrame(data)

## Pretvaranje kategorijskih podataka u numeričke vrijednosti
Kako bi modeli mogli raditi sa ovim podacima, kategorijske varijable se pretvaraju u numeričke vrijednosti.

1. df.columns:
df.columns je atribut Pandas DataFrame objekta koji sadrži listu svih naziva kolona u DataFrame-u. U ovom slučaju, ovo će vratiti listu naziva svih kolona u DataFrame-u df.
2. Petlja for column in df.columns:
Ova petlja prolazi kroz svaku kolonu u DataFrame-u. Za svaku kolonu, izvršava se kod unutar petlje. Na primjer, u prvom prolazu, column će biti "Ukupan rizik", zatim "Vremenski horizont", itd.
3. df[column]:
df[column] pristupa podacima u trenutnoj koloni. Dakle, u prvom prolazu, to je cijela kolona "Ukupan rizik".
4. astype("category"):
Ova funkcija u Pandas-u koristi se za pretvaranje podataka u kategorijski tip podataka (category). Kategorijski podaci su varijable koje imaju ograničen broj mogućih vrijednosti. U ovom slučaju, na primjer, kolona "Ukupan rizik" može imati vrijednosti "nizak" i "visok", koje su kategorije. Pretvaranje u category tip omogućava brže manipuliranje podacima, jer Pandas zna da su vrijednosti ograničene i specifične za tu kolonu.
5. cat.codes:
cat.codes je atribut koji se koristi za pretvaranje kategorijskih vrijednosti u numeričke kodove. Svakoj kategoriji se dodjeljuje jedinstveni broj. Na primjer: "nizak" može biti 0, "visok" može biti 1. Ovaj proces omogućava da kategorijski podaci (npr. tekstualne vrijednosti) budu pretvoreni u numeričke kodove koje mogu koristiti algoritmi mašinskog učenja.
6. df[column] = ...:
Ovdje, rezultati pretvaranja kategorijskih podataka u numeričke (putem cat.codes) zamjenjuju originalne podatke u DataFrame-u. Na primjer, ako je kolona "Ukupan rizik" imala vrijednosti ["nizak", "visok", "nizak"], nakon pretvorbe, ta kolona će imati vrijednosti [0, 1, 0].

In [5]:
for column in df.columns:
    df[column] = df[column].astype("category").cat.codes

## Odvajanje ulaznih i izlaznih varijabli
Ulazne karakteristike (X) se izdvajaju od ciljne varijable (y).

Ovaj kod služi za odvajanje ulaznih (karakteristika) i izlaznih (ciljnih) varijabli, što je osnovni korak u pripremi podataka za modele mašinskog učenja. 

1. X = df.drop(columns=["Realizovati?"])
   df.drop(columns=["Realizovati?"]):
Funkcija drop u Pandas-u koristi se za uklanjanje određenih kolona (ili redova) iz DataFrame-a. Ovdje se specificira columns=["Realizovati?"], što znači da želimo ukloniti kolonu "Realizovati?" iz DataFrame-a df. Nakon što je kolona "Realizovati?" uklonjena, preostale kolone (kao što su "Ukupan rizik", "Vremenski horizont", itd.) postaju ulazne karakteristike za naš model. 
Varijabla X će sadržavati sve kolone iz DataFrame-a df osim kolone "Realizovati?", jer ova kolona predstavlja ciljnu varijablu koju želimo predvidjeti, a ne ulazne karakteristike.

2. y = df["Realizovati?"]
   df["Realizovati?"]:
Ovdje se selektuje samo kolona "Realizovati?" iz DataFrame-a df, koja predstavlja izlaznu varijablu (ciljnu varijablu). Ova kolona sadrži informacije koje želimo da model nauči predviđati (u ovom slučaju, hoće li nešto biti "realizovano" ili ne, tj. odgovor "da" ili "ne").

Varijabla y će biti DataFrame (ili Pandas Series) koji sadrži samo vrijednosti iz kolone "Realizovati?", koje će služiti kao odgovori koje model pokušava predvidjeti na osnovu ulaznih karakteristika.

3. Odvajanje ulaznih i izlaznih varijabli
Ulazne varijable (X): Ove varijable su karakteristike koje model koristi za donošenje predikcija. U ovom slučaju, to su sve kolone osim "Realizovati?" i predstavljaju informacije koje model koristi da nauči obrasce u podacima.

Izlazna varijabla (y): Ovo je ciljni odgovor koji želimo da naš model predviđa. U ovom slučaju, to je kolona "Realizovati?", koja sadrži vrijednosti "da" ili "ne" i označava hoće li nešto biti realizovano ili ne.

In [6]:
X = df.drop(columns=["Realizovati?"])
y = df["Realizovati?"]

## Podjela podataka na trening i test set
Podaci se dijele na skup za obuku i skup za testiranje pomoću funkcije train_test_split.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Implementacija Naivnog Bayesa
Treniranje modela Naivnog Bayesa i generiramo predikcije.

1. nb_model = CategoricalNB()
Ovdje se kreira instanca Naivnog Bayesovog - klasifikatora za kategorizirane podatke.
CategoricalNB je specifična varijanta Naivnog Bayesovog algoritma koja je optimizirana za kategorizirane (diskretne) podatke.  Korištenje CategoricalNB je prikladno kada su ulazne karakteristike (u ovom slučaju, varijable u X_train) kategorizirane, a ne kontinuirane.

2. nb_model.fit(X_train, y_train)
Ova linija trenira (fituje) model koristeći trening skup podataka.
X_train predstavlja ulazne podatke, tj. karakteristike koje model koristi za predviđanje (u ovom slučaju to su varijable kao što su "Ukupan rizik", "Vremenski horizont", itd.).
y_train predstavlja ciljne varijable koje želimo da model predviđa (u ovom slučaju, "Realizovati?").
Funkcija fit koristi ove podatke za učenje modela, odnosno za izračunavanje parametara modela koji najbolje odgovaraju trening podacima.

3. nb_predictions = nb_model.predict(X_test)
Ovdje se koristi trenirani model da napravimo predikcije na temelju testnog skupa podataka. X_test sadrži nove podatke koji nisu korišteni u fazi treniranja i na kojima model treba da donese predikcije. Testni skup podataka služi za evaluaciju sposobnosti modela da generalizira na nove, neviđene podatke. Funkcija predict generira predikcije za svaki red u testnom skupu podataka X_test, a predikcije se spremaju u varijablu nb_predictions.

4. nb_accuracy = accuracy_score(y_test, nb_predictions)
Ova linija koristi funkciju accuracy_score iz biblioteke sklearn.metrics kako bi izračunala tačnost modela.
y_test sadrži stvarne ciljne vrijednosti (tj. stvarne odgovore iz testnog skupa podataka koji predstavljaju "Realizovati?").
nb_predictions sadrži predikcije koje je model napravio za testne podatke.
Funkcija accuracy_score upoređuje stvarne vrijednosti (y_test) i predikcije (nb_predictions) i izračunava postotak tačnih predikcija, što nam daje mjerenje tačnosti modela. Ovaj postotak predstavlja tačnost modela, odnosno koliko je predikcija ispravno klasificirano.

In [8]:
nb_model = CategoricalNB()
nb_model.fit(X_train, y_train)
nb_predictions = nb_model.predict(X_test)

nb_accuracy = accuracy_score(y_test, nb_predictions)

## Implementacija stabla odlučivanja
Trenira se model stabla odlučivanja i generiraju predikcije.

1. tree_model = DecisionTreeClassifier(random_state=42)
Ovdje se kreira instanca DecisionTreeClassifier, što je model temeljen na algoritmu za odlučujuća stabla.
random_state=42 se koristi kako bi se osigurala reproduktivnost rezultata. Korištenje fiksne vrijednosti za random seed osigurava da će podaci biti podijeljeni na isti način svaki put kad pokrenemo kod, čime se omogućuje dosljednost u rezultatima.

2. tree_model.fit(X_train, y_train)
Ovdje se trenira model na temelju trening podataka.
X_train predstavlja ulazne karakteristike, odnosno varijable koje model koristi za donošenje odluka (npr. "Ukupan rizik", "Vremenski horizont", itd.).
y_train je ciljna varijabla (odgovor koji želimo da model predviđa), u ovom slučaju "Realizovati?".
Funkcija fit koristi ove podatke za "učenje" modela, odnosno za kreiranje stabla odluka koje najbolje objašnjava odnose između ulaznih karakteristika i ciljne varijable.

3. tree_predictions = tree_model.predict(X_test)
Nakon što je model treniran, koristi se funkcija predict da generiramo predikcije na temelju novih podataka, koji nisu bili uključeni u trening (testni podaci).
X_test je testni skup podataka koji sadrži ulazne karakteristike za koje želimo dobiti predikcije. Ovaj skup podataka služi za evaluaciju modela.
Funkcija predict koristi naučena pravila i odluke iz stabla da predvidi vrijednost ciljne varijable za svaki red u X_test. Predikcije će biti pohranjene u varijabli tree_predictions.

4. tree_accuracy = accuracy_score(y_test, tree_predictions)
Ovdje se koristi funkcija accuracy_score iz biblioteke sklearn.metrics da izračunamo tačnost modela.
y_test predstavlja stvarne ciljne vrijednosti (stvarni odgovori na temelju kojih ćemo ocjenjivati tačnost modela).
tree_predictions predstavlja predikcije koje je model napravio za testne podatke.
Funkcija accuracy_score uspoređuje stvarne ciljeve (y_test) i predikcije (tree_predictions) i izračunava postotak tačnih predikcija, što daje tačnost modela. Tačnost označava koliko je model bio uspješan u pravilnom klasificiranju testnih podataka.

In [9]:
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)
tree_predictions = tree_model.predict(X_test)

tree_accuracy = accuracy_score(y_test, tree_predictions)

## Prikaz pravila stabla odlučivanja
Izvodimo pravila generisana stablom odlučivanja.

- export_text(tree_model, feature_names=list(X.columns))
- tree_model: Ovo je već trenirani model za odlučujuće stablo (DecisionTreeClassifier), koji sadrži sve informacije o strukturi stabla i pravilima donošenja odluka.
- X.columns: Ovo je lista imena svih ulaznih karakteristika (atributa) u trening skupu podataka. Na primjer, to mogu biti imena kao što su "Ukupan rizik", "Vremenski horizont", itd.
- list(X.columns): Funkcija list() pretvara imena kolona u listu koja se zatim prosljeđuje funkciji export_text. To omogućava da se za svaku karakteristiku prikaže njeno ime u tekstualnom prikazu pravila stabla.
- export_text funkcija prikazuje odluke koje je model donio u obliku razumljivog teksta. Pravila stabla su predstavljena tako da možete vidjeti kako su ulazne karakteristike korištene za donošenje odluka i kako se prelazi od korijena do lišća stabla.

In [10]:
tree_rules = export_text(tree_model, feature_names=list(X.columns))

## Rezultati
Prikazuje tačnost oba modela i pravila stabla odlučivanja.

In [11]:
print("\n----------------------------------")
print("Rezultati klasifikacije:\n")
print(f"Naivni Bayes tačnost: {nb_accuracy:.2f}")
print(f"Stablo odlučivanja tačnost: {tree_accuracy:.2f}")
print("\n----------------------------------")
print("Pravila stabla odlučivanja:")
print(tree_rules)
print("----------------------------------")


----------------------------------
Rezultati klasifikacije:

Naivni Bayes tačnost: 0.33
Stablo odlučivanja tačnost: 0.33

----------------------------------
Pravila stabla odlučivanja:
|--- Drustvena korist <= 0.50
|   |--- class: 1
|--- Drustvena korist >  0.50
|   |--- Ukupan rizik <= 0.50
|   |   |--- Vremenski horizont <= 2.50
|   |   |   |--- Vremenski horizont <= 1.50
|   |   |   |   |--- Vremenski horizont <= 0.50
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- Vremenski horizont >  0.50
|   |   |   |   |   |--- class: 0
|   |   |   |--- Vremenski horizont >  1.50
|   |   |   |   |--- class: 1
|   |   |--- Vremenski horizont >  2.50
|   |   |   |--- class: 0
|   |--- Ukupan rizik >  0.50
|   |   |--- class: 0

----------------------------------
